In [1]:
!where python



C:\ProgramData\Anaconda3\envs\mini-projet\python.exe
C:\Users\RACHID AHSOUNE\AppData\Local\Programs\Python\Python37\python.exe
C:\Users\RACHID AHSOUNE\AppData\Local\Microsoft\WindowsApps\python.exe


In [2]:
!jupyter kernelspec list --json

{
  "kernelspecs": {
    "anaconda-mini-projet": {
      "resource_dir": "C:\\Users\\RACHID AHSOUNE\\AppData\\Roaming\\jupyter\\kernels\\anaconda-mini-projet",
      "spec": {
        "argv": [
          "C:\\ProgramData\\Anaconda3\\envs\\mini-projet\\python.exe",
          "-m",
          "ipykernel_launcher",
          "-f",
          "{connection_file}"
        ],
        "env": {},
        "display_name": "Anaconda (mini-projet)",
        "language": "python",
        "interrupt_mode": "signal",
        "metadata": {}
      }
    },
    "python3": {
      "resource_dir": "c:\\programdata\\anaconda3\\envs\\mini-projet\\share\\jupyter\\kernels\\python3",
      "spec": {
        "argv": [
          "python",
          "-m",
          "ipykernel_launcher",
          "-f",
          "{connection_file}"
        ],
        "env": {},
        "display_name": "Python 3",
        "language": "python",
        "interrupt_mode": "signal",
        "metadata": {}
      }
    }
  }
}


In [3]:
!python --version

Python 3.6.9 :: Anaconda, Inc.


## Question Answering

### Download and Prepare Data

### Prepare Data

In [8]:
import json 

def read_data(file_path, max_context_size = 100):
  # Read dataset
  with open(file_path,encoding="utf-8") as f:
      data = json.load(f)

  contexts = []
  questions = []
  answers = []
  labels = []
  
  for i in range(len(data['data'])):

    paragraph_object = data['data'][i]["paragraphs"]
    
    for j in range(len(paragraph_object)):

      context_object = paragraph_object[j]
      context_text = context_object['context']

      if len(context_text.split()) > max_context_size:
        continue
      for k in range(len(context_object['qas'])):

        question_object = context_object['qas'][k]
        question_text = question_object['question']
        
        answer_object = question_object['answers'][0]
        answer_text = answer_object['text']
        answer_start = answer_object['answer_start']
        answer_end = answer_start + len(answer_text)

        answer_start = len(context_text[:answer_start].split())
        answer_end = answer_start + len(answer_text.split())
        if answer_end >= max_context_size:
          answer_end = max_context_size -1
        labels.append([answer_start, answer_end])
        questions.append(question_text)
        contexts.append(context_text)
        answers.append(answer_text)
        
  with open('train_contexts.txt', 'w',encoding="utf-8") as f:
    f.write(('\n').join(contexts))
  with open('train_questions.txt', 'w' ,encoding="utf-8") as f:
    f.write(('\n').join(questions))
  # with open('questAnswers.txt',"w") as file:
  #   file.write(('\n').join(questAnswers))
  return {'qas':questions, 'ctx':contexts, 'ans':answers, 'lbl':labels}

In [9]:
train_data = read_data('test-context-ar-question-ar.json')


In [10]:
for i in range(10):
  print(train_data['qas'][i])
  print(train_data['ctx'][i])
  print(train_data['ans'][i])
  print("==============")

ما الذي جعل شريط الاختبار للطائرة؟
بحيرة جرووم كانت تستخدم للقصف المدفعي والتدريب علي المدفعية خلال الحرب العالمية الثانية، ولكن تم التخلي عنها بعد ذلك حتى نيسان / أبريل 1955، عندما تم اختياره من قبل فريق لوكهيد اسكنك كموقع مثالي لاختبار لوكهيد يو-2 - 2 طائرة التجسس. قاع البحيرة قدم الشريط المثالية التي يمكن عمل اختبارات الطائرات المزعجة، ارتفاع سلسلة جبال وادي الإيمجرانت ومحيط NTS يحمي موقع الاختبار من أعين المتطفلين والتدخل الخارجي.
قاع البحيرة
من كان يرافق طائرة يو -2 عند التسليم؟
شيدت لوكهيد قاعدة مؤقتة في الموقع، ثم عرفت باسم الموقع الثاني أو "المزرعة"، التي تتألف من أكثر بقليل من بضعة مخابئ، وحلقات عمل ومنازل متنقلة لفريقها الصغير. في ثلاثة أشهر فقط شيد مدرج طوله 5000  ودخل الخدمة بحلول تموز / يوليو 1955. حصلت المزرعة على تسليم أول يو 2 في 24 يوليو، 1955 من بوربانك على سي 124 جلوب ماستر الثاني طائرة شحن، يرافقه فنيي وكهيد على دي سي 3. انطلق أول يو - 2 من الجرووم في 4 أغسطس، 1955. بدأت عمليات تحليق أسطول يو 2 تحت سيطرة وكالة المخابرات المركزية الأمريكية في الأجواء السوفياتية بحلول

### Imports

In [11]:
import re
import nltk
import time
import numpy as np
import tkseem as tk
from flask import Flask, render_template, request
# from flask_ngrok import run_with_ngrok
# import pyarabic.araby as araby
# from __future__ import unicode_literals
import tensorflow as tf
import keras.models
import matplotlib.ticker as ticker
import matplotlib.pyplot as plt

### Tokenization

In [7]:
qa_tokenizer = tk.WordTokenizer()
qa_tokenizer.train('train_questions.txt')
# print(qa_tokenizer)


cx_tokenizer = tk.WordTokenizer()

cx_tokenizer.train('train_contexts.txt')
# print(cx_tokenizer)


train_inp_data = qa_tokenizer.encode_sentences(train_data['qas'])
# print(train_inp_data)

train_tar_data = cx_tokenizer.encode_sentences(train_data['ctx'])
# print(train_tar_data)
train_tar_lbls = train_data['lbl']
train_inp_data.shape, train_tar_data.shape

Training WordTokenizer ...


UnicodeDecodeError: 'charmap' codec can't decode byte 0x81 in position 84: character maps to <undefined>

### Create Dataset

In [12]:
BATCH_SIZE = 64
BUFFER_SIZE = len(train_inp_data)

dataset = tf.data.Dataset.from_tensor_slices((train_inp_data, train_tar_data, train_tar_lbls)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)

NameError: name 'train_inp_data' is not defined

### Create Encoder and Decoder

In [ ]:
class Encoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, enc_units, batch_sz):
        super(Encoder, self).__init__()
        self.batch_sz = batch_sz
        self.enc_units = enc_units
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = tf.keras.layers.GRU(self.enc_units,
                                       recurrent_initializer='glorot_uniform')

    def call(self, x, hidden):
        x = self.embedding(x)
        output = self.gru(x, initial_state = hidden)
        return output

    def initialize_hidden_state(self):
        return tf.zeros((self.batch_sz, self.enc_units))
  
class Decoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, dec_units, output_sz):
        super(Decoder, self).__init__()
        self.dec_units = dec_units
        self.embedding_dim = embedding_dim
        self.output_sz = output_sz
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = tf.keras.layers.GRU(self.dec_units,
                                       return_sequences=False,
                                       recurrent_initializer='glorot_uniform')
        self.fc11 = tf.keras.layers.Dense(embedding_dim)
        self.fc12 = tf.keras.layers.Dense(output_sz)

        self.fc21 = tf.keras.layers.Dense(embedding_dim)
        self.fc22 = tf.keras.layers.Dense(output_sz)
        
    def call(self, x, hidden):
        x = self.embedding(x)
        x = self.gru(x, initial_state = hidden)
        x1 = self.fc11(x)
        x2 = self.fc21(x)

        x1 = self.fc12(x1)
        x2 = self.fc22(x2)
        return [x1, x2]

def loss_fn(true, pred):
  cross_entropy = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
  return (cross_entropy(true[:,0:1], pred[0]) + cross_entropy(true[:,1:2], pred[1]))/2

### Training

In [ ]:
units = 1024
embedding_dim = 256
max_length_inp = train_inp_data.shape[1]
max_length_tar = train_tar_data.shape[1]
vocab_tar_size = cx_tokenizer.vocab_size
vocab_inp_size = qa_tokenizer.vocab_size
steps_per_epoch = len(train_inp_data) // BATCH_SIZE
decoder = Decoder(vocab_tar_size, embedding_dim, units, max_length_tar)
encoder = Encoder(vocab_inp_size, embedding_dim, units, BATCH_SIZE)

optim = tf.optimizers.Adam()

In [ ]:
checkpoint_path = "./checkpoints/train"
ckpt = tf.train.Checkpoint(encoder=encoder,
                           decoder=decoder,
                           optimizer = optim)
ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep=5)

In [ ]:
start_epoch = 0
if ckpt_manager.latest_checkpoint:
  start_epoch = int(ckpt_manager.latest_checkpoint.split('-')[-1])

In [ ]:
epochs = 60
# print(start_epoch)
for epoch in range(start_epoch,epochs):
  enc_hidden = encoder.initialize_hidden_state()
  epoch_loss = 0
  
  for idx, (inp, tar, true) in enumerate(dataset):
    with tf.GradientTape() as tape:
        hidden = encoder(inp, enc_hidden)
        pred = decoder(tar, hidden)
        loss  = loss_fn(true, pred)
    variables = decoder.trainable_variables + encoder.trainable_variables
    gradients = tape.gradient(loss, variables)
    optim.apply_gradients(zip(gradients, variables))
    epoch_loss += loss.numpy()
    if epoch % 5 == 0:
      ckpt_manager.save()
  print(f"Epoch {epoch} loss: {epoch_loss/steps_per_epoch:.3f}")


### evaluation

In [ ]:

encoder.save_weights("encoder_weights",save_format='tf')
decoder.save_weights("decoder_weights",save_format='tf')

In [ ]:
def answer(question_txt,context_txt):
    question = qa_tokenizer.encode_sentences([question_txt], out_length = max_length_inp)
    context = cx_tokenizer.encode_sentences([context_txt], out_length = max_length_tar)
    question = tf.convert_to_tensor(question)
    context = tf.convert_to_tensor(context)
    
    # encoder.load_weights("/content/drive/MyDrive/Colab Notebooks/encoder_weights")
    # decoder.load_weights("/content/drive/MyDrive/Colab Notebooks/decoder_weights")

    hidden = [tf.zeros((1, units))]
    enc_hidden = encoder(question, hidden)
    pred = decoder(context, enc_hidden)

    start = tf.argmax(pred[0], axis = -1).numpy()[0]
    end = tf.argmax(pred[1], axis = -1).numpy()[0]
    
    if start >= len(context_txt.split()):
      start = len(context_txt.split()) - 1
    if end >= len(context_txt.split()):
      end = len(context_txt.split()) - 1
    
    # if one word prediction
    if end == start:
      end += 1
    answer_txt = (' ').join(context_txt.split()[start:end])
    return answer_txt
    

In [ ]:

def response(userQuestion):
    qst=open('train_questions.txt', 'r')
    cnt=open('train_contexts.txt', 'r')
    # word = u"الْعَرَبِيَّةُ"

    # if araby.is_arabicword(word):
    #    print("valid arabic")
    # else: 
    #    print ("invalid arabic word")
    # araby.strip_tashkeel(word)
    # araby.strip_tatweel(word)
    # print(word)
    res=""
    question = qst.readlines()
    for i in range(len(question)):
       question[i].replace("\n",'')
      #  print(question[i])
       if userQuestion in question[i]:
             cont=train_data['ctx'][i]
            #  print(cont)
             res=answer(userQuestion,cont)
             break    
    print(res)
    return res

In [ ]:
response("من كان يرافق طائرة يو؟")

In [ ]:
app = Flask(__name__)
# run_with_ngrok(app)

@app.route("/")
def home():
    return render_template("index.html")

@app.route("/get")
def get_bot_reponse():
    userText = request.args.get('msg')
    print(userText)
    print(str(response(userText)))
    return str(response(userText))
if __name__ == '__main__':
  app.run()
